### 给定一个参考时间点，根据每张卡距离该参考时间点最近的3个月交易/刷卡数据，以及该参考时间点之后两个月的交易/刷卡数据来计算每张卡（主人）的忠诚度。除此之外，还有额外消费商店相关的数据，如商店分类、商店交易量等信息。
************************************************************************************************************************************************
2018.12.07: 审题的结果是一个时间序列相关的问题。但是看到圈内都用lightgbm做了回归预测，所以个人感觉有空间提升。
************************************************************************************************************************************************
数据的理解：
* train：card_id, first_purchase_date, f1, f2, f3, target(loyalty)
* test: card_id, first_purchase_date, f1, f2, f3, target(loyalty)
* history/new: card_id, month_lag, purchase_date, authorized_flag, category_3, installments, category_1, merchant_category_id, subsector_id, merchant_id, purchase_amount, city_id, state_id, category_2 
* merchant: merchant_id, merchant_group_id, merchant_category_id, subsector_id, numerical_1, numerical_2, category_1, most_recent_sales_range, most_recent_purchases_range, avg_sales_lag3, avg_purchases_lag3, active_months_lag3, avg_sales_lag6 , ,avg_purchases_lag6, active_months_lag6, avg_sales_lag12, avg_purchases_lag12, active_months_lag12, category_4, city_id, state_id, category_2
************************************************************************************************************************************************

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.listdir("../dataset"))

['Data_Dictionary.xlsx', 'new_merchant_transactions.csv', 'test.csv', 'tmp_data.csv', 'merchants.csv', 'historical_transactions.csv', 'train.csv', 'sample_submission.csv']


In [2]:
# common functions
def binaries(value):
    if value == 'Y':
        return 1
    else:
        return 0

from datetime import datetime
from dateutil import rrule
def get_month_span(month):
    baseline = '2018-02'
    x = datetime.strptime(baseline, '%Y-%m')
    y = datetime.strptime(month, '%Y-%m')
    months = rrule.rrule(rrule.MONTHLY, dtstart=y, until=x).count()
    return months

def get_day_span(date):
    baseline = '2018-03-01 00:00:00'
    x = datetime.strptime(baseline, '%Y-%m-%d %H:%M:%S')
    y = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
    return (x - y).days

def get_days_from_purchase(date):
    baseline = '2018-03-01 00:00:00'
    x = datetime.strptime(baseline, '%Y-%m-%d %H:%M:%S')
    days = x - date
    return days

#from collections import Counter
#s = hist_data['category_2'].tolist()
#word_counts = Counter(s)
#top_three = word_counts.most_common(3)
#print(top_three)
#hist_data.loc[hist_data['merchant_id'] == 'M_ID_00a6ca8a8a']['merchant_id'].count()
# history transaction data
#f_history = open('../dataset/historical_transactions.csv')
#history_data_chunk = pd.read_csv(f_history, iterator=True)
#loop = True
#chunkSize = 100000
#chunks = []
#while loop:
#    try:
#        chunk = history_data_chunk.get_chunk(chunkSize)
#        chunks.append(chunk)
#    except StopIteration:
#        loop = False
#history_transaction = pd.concat(chunks, ignore_index=True)

### merchant data

In [3]:
# merchant data
merchants_data = pd.read_csv("../dataset/merchants.csv")
# fillna
avg_sales_lag3_mean = merchants_data['avg_sales_lag3'].mean()
merchants_data['avg_sales_lag3'].fillna(avg_sales_lag3_mean, inplace=True)
avg_sales_lag6_mean = merchants_data['avg_sales_lag6'].mean()
merchants_data['avg_sales_lag6'].fillna(avg_sales_lag6_mean, inplace=True)
avg_sales_lag12_mean = merchants_data['avg_sales_lag12'].mean()
merchants_data['avg_sales_lag12'].fillna(avg_sales_lag12_mean, inplace=True)
merchants_data['category_2'].fillna(0, inplace=True)
# change date type
merchants_data['merchant_id'] = merchants_data['merchant_id'].astype('category')
merchants_data['merchant_group_id'] = merchants_data['merchant_group_id'].astype('category')
merchants_data['merchant_category_id'] = merchants_data['merchant_category_id'].astype('category')
merchants_data['subsector_id'] = merchants_data['subsector_id'].astype('category')
merchants_data['category_1'] = merchants_data['category_1'].astype('category')
merchants_data['most_recent_sales_range'] = merchants_data['most_recent_sales_range'].astype('category')
merchants_data['most_recent_purchases_range'] = merchants_data['most_recent_purchases_range'].astype('category')
merchants_data['category_2'] = merchants_data['category_2'].astype('category')
merchants_data['category_4'] = merchants_data['category_4'].apply(binaries).astype('bool')
merchants_data['city_id'] = merchants_data['city_id'].astype('category')
merchants_data['state_id'] = merchants_data['state_id'].astype('category')

### construct features
# group: merchant number of each group
# category: merchant number of each category
# subsector: merchant number of each subsector
# city: merchant number of each city
# state: merchant number of each state
merchants_group = {k: v for k, v in merchants_data['merchant_group_id'].value_counts().iteritems()}
merchants_category = {k: v for k, v in merchants_data['merchant_category_id'].value_counts().iteritems()}
merchants_subsector = {k: v for k, v in merchants_data['subsector_id'].value_counts().iteritems()}
merchants_city = {k: v for k, v in merchants_data['city_id'].value_counts().iteritems()}
merchants_state = {k: v for k, v in merchants_data['state_id'].value_counts().iteritems()}

def count_group(x):
    return merchants_group[x]
def count_cat(x):
    return merchants_category[x]
def count_subset(x):
    return merchants_subsector[x]
def count_city(x):
    return merchants_city[x]
def count_state(x):
    return merchants_state[x]

merchants_data['group_count'] = merchants_data['merchant_group_id'].apply(count_group).astype(np.int64)
merchants_data['cat_count'] = merchants_data['merchant_category_id'].apply(count_cat).astype(np.int64)
merchants_data['subset_count'] = merchants_data['subsector_id'].apply(count_subset).astype(np.int64)
merchants_data['city_count'] = merchants_data['city_id'].apply(count_city).astype(np.int64)
merchants_data['state_count'] = merchants_data['state_id'].apply(count_state).astype(np.int64)

# rename features
merchants_data = merchants_data.rename(columns = {'merchant_group_id':'m_gid', 
                                                  'merchant_category_id':'m_cid', 
                                                  'subsector_id':'m_sid', 
                                                  'numerical_1':'m_num1', 
                                                  'numerical_2':'m_num2', 
                                                  'category_1':'m_cat1', 
                                                  'category_2':'m_cat2',
                                                  'category_4':'m_cat4',
                                                  'most_recent_sales_range':'m_mrsr', 
                                                  'most_recent_purchases_range':'m_mrpr', 
                                                  'avg_sales_lag3':'m_asl3', 
                                                  'avg_purchases_lag3':'m_apl3', 
                                                  'active_months_lag3':'m_aml3', 
                                                  'avg_sales_lag6':'m_asl6', 
                                                  'avg_purchases_lag6':'m_apl6', 
                                                  'active_months_lag6':'m_aml6',
                                                  'avg_sales_lag12':'m_asl12', 
                                                  'avg_purchases_lag12':'m_apl12', 
                                                  'active_months_lag12':'m_aml12', 
                                                  'city_id':'m_city', 
                                                  'state_id':'m_state', 
                                                  'group_count':'m_group_count', 
                                                  'cat_count':'m_cat_count', 
                                                  'subset_count':'m_subset_count', 
                                                  'city_count':'m_city_count', 
                                                  'state_count':'m_state_count', 
                                                  'merchant_id':'m_id'})
merchants_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 334696 entries, 0 to 334695
Data columns (total 27 columns):
m_id              334696 non-null category
m_gid             334696 non-null category
m_cid             334696 non-null category
m_sid             334696 non-null category
m_num1            334696 non-null float64
m_num2            334696 non-null float64
m_cat1            334696 non-null category
m_mrsr            334696 non-null category
m_mrpr            334696 non-null category
m_asl3            334696 non-null float64
m_apl3            334696 non-null float64
m_aml3            334696 non-null int64
m_asl6            334696 non-null float64
m_apl6            334696 non-null float64
m_aml6            334696 non-null int64
m_asl12           334696 non-null float64
m_apl12           334696 non-null float64
m_aml12           334696 non-null int64
m_cat4            334696 non-null bool
m_city            334696 non-null category
m_state           334696 non-null category
m_cat2 

### historical data

In [4]:
# load data
hist_data = pd.read_csv('../dataset/historical_transactions.csv')
# fillna
hist_data['category_2'].fillna(1.0, inplace=True)
hist_data['category_3'].fillna('A', inplace=True)
hist_data['merchant_id'].fillna('M_ID_00a6ca8a8a', inplace=True)
# change data type
hist_data['authorized_flag'] = hist_data['authorized_flag'].apply(binaries).astype('bool')
hist_data['card_id'] = hist_data['card_id'].astype('category')
hist_data['city_id'] = hist_data['city_id'].astype('category')
hist_data['category_1'] = hist_data['category_1'].apply(binaries).astype('bool')
hist_data['category_2'] = hist_data['category_2'].astype('int').astype('category')
hist_data['category_3'] = hist_data['category_3'].astype('category')
hist_data['merchant_category_id'] = hist_data['merchant_category_id'].astype('category')
hist_data['merchant_id'] = hist_data['merchant_id'].astype('category')
hist_data['purchase_date'] = pd.to_datetime(hist_data['purchase_date'])
hist_data['state_id'] = hist_data['state_id'].astype('category')
hist_data['subsector_id'] = hist_data['subsector_id'].astype('category')

# construct features
# purchase_date: year, month, day
# card_id: agg total numbers
hist_data['purchase_date'] = hist_data['purchase_date'].apply(get_days_from_purchase)
card_purchase_count = {k: v for k, v in hist_data['card_id'].value_counts().iteritems()}
def count_purchase(x):
    return card_purchase_count[x]
hist_data['card_count'] = hist_data['card_id'].apply(count_purchase).astype(np.int64)
hist_data.drop(columns=['purchase_date'], inplace=True)
# rename features
hist_data = hist_data.rename(columns={'card_id':'c_id', 
                                      'authorized_flag': 'c_af',
                                      'city_id': 'c_city', 
                                      'state_id': 'c_state',
                                      'subsector_id': 'c_ms',
                                      'category_1':'c_cat1', 
                                      'category_2':'c_cat2', 
                                      'category_3':'c_cat3', 
                                      'installments':'c_inst', 
                                      'merchant_category_id':'c_mcid', 
                                      'merchant_id':'m_id', 
                                      'month_lag': 'c_ml', 
                                      'purchase_amount':'c_pa', 
                                      'card_count':'c_cc', 
                                      'purchase_ref_days':'c_prd'})

### train data

In [5]:
# train data
train_data = pd.read_csv("../dataset/train.csv")
# change data type
train_data['first_active_month'] = train_data['first_active_month'].apply(get_month_span)
train_data['card_id'] = train_data['card_id'].astype('category')
train_data['feature_1'] = train_data['feature_1'].astype('category')
train_data['feature_2'] = train_data['feature_2'].astype('category')
train_data['feature_3'] = train_data['feature_3'].astype('category')
# rename feature
train_data = train_data.rename(columns={'first_active_month':'t_days', 
                                        'card_id':'c_id', 
                                        'feature_1':'t_f1', 
                                        'feature_2':'t_f2', 
                                        'feature_3':'t_f3'})

### test data

In [6]:
# test data
test_data = pd.read_csv("../dataset/test.csv")
# fillna
test_data['first_active_month'].fillna('2018-02', inplace=True)
# change data type
test_data['first_active_month'] = test_data['first_active_month'].apply(get_month_span)
test_data['card_id'] = test_data['card_id'].astype('category')
test_data['feature_1'] = test_data['feature_1'].astype('category')
test_data['feature_2'] = test_data['feature_2'].astype('category')
test_data['feature_3'] = test_data['feature_3'].astype('category')
test_data = test_data.rename(columns={'first_active_month':'t_days', 
                                        'card_id':'c_id', 
                                        'feature_1':'t_f1', 
                                        'feature_2':'t_f2', 
                                        'feature_3':'t_f3'})

### merge data

In [7]:
tmp = pd.merge(train_data, hist_data, how='left', on='c_id')
tmp = pd.merge(tmp, merchants_data, how='left', on='m_id')
tmp['c_id'] = tmp['c_id'].astype('category')
tmp['m_id'] = tmp['m_id'].astype('category')

### train model

In [8]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

/anaconda3/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [13]:
y_tmp = tmp['target'].values
X_tmp = tmp.drop(columns=['target', 'c_id', 'm_id'], axis=1)
#X_train, X_test, y_train, y_test = train_test_split(X_tmp, y_tmp, test_size=0.4, random_state=0)

In [14]:
d_train = lgb.Dataset(X_tmp, y_tmp)
watchlist = [d_train]

In [15]:
params = {
        'objective': 'regression',
        'boosting': 'gbdt',
        'learning_rate': 0.5 ,
        'verbose': 0,
        'num_leaves': 2**8,
        'max_depth': 20,
        'metric' : 'rmse'
}
model = lgb.train(params, train_set=d_train, num_boost_round=60, valid_sets=watchlist, verbose_eval=5)

[5]	training's rmse: 3.38946
[10]	training's rmse: 3.30439
[15]	training's rmse: 3.23076
[20]	training's rmse: 3.18346
[25]	training's rmse: 3.14676
[30]	training's rmse: 3.11005
[35]	training's rmse: 3.07428
[40]	training's rmse: 3.04652
[45]	training's rmse: 3.01365
[50]	training's rmse: 2.98229
[55]	training's rmse: 2.95676
[60]	training's rmse: 2.92729


In [16]:
test = pd.merge(test_data, hist_data, how='left', on='c_id')
test = pd.merge(test, merchants_data, how='left', on='m_id')
test['c_id'] = test['c_id'].astype('category')
test['m_id'] = test['m_id'].astype('category')
ids = test['c_id'].values
test_set = test.drop(columns=['c_id', 'm_id'], axis=1)

In [17]:
test_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11600086 entries, 0 to 11600085
Data columns (total 42 columns):
t_days            int64
t_f1              category
t_f2              category
t_f3              category
c_af              bool
c_city            category
c_cat1            bool
c_inst            int64
c_cat3            category
c_mcid            category
c_ml              int64
c_pa              float64
c_cat2            category
c_state           category
c_ms              category
c_cc              int64
m_gid             category
m_cid             category
m_sid             category
m_num1            float64
m_num2            float64
m_cat1            category
m_mrsr            category
m_mrpr            category
m_asl3            float64
m_apl3            float64
m_aml3            int64
m_asl6            float64
m_apl6            float64
m_aml6            int64
m_asl12           float64
m_apl12           float64
m_aml12           int64
m_cat4            bool
m_city   

In [18]:
predictions = model.predict(test_set)
subm = pd.DataFrame()
subm['card_id'] = ids
subm['target'] = predictions
subm.to_csv('../dataset/submission.csv.gz', compression = 'gzip', index=False, float_format = '%.5f')

print('Done!')

Done!


In [36]:
target = subm.groupby('card_id')['target'].max()

In [37]:
r = pd.DataFrame({'card_id':target.index, 'target':target.values})

In [38]:
r.to_csv('../dataset/submission.csv', index=False, float_format = '%.5f')
print('Done!')

Done!
